In [1]:
#hbo data- figure out how to scroll down
#more movies from rotten tomatoes from last couple of years***********************
#amazon prime video
#data frame with movies and which service they can be found on
#get everything in format to use with IMDB API (probably using dataframe)
#remove Les, la, at end of titles


#with different variables
    #-genre
    #-words from plot synopsis
    #-director
    #-year
    #-actors
    #-rating from different viewers(how are we going to define which movies people will like? how do we test the model?)
    
#when you type movie, where can they be found in catalogue?
#out of entire selection of movies, which is most similar or enjoyed by similar reviewers? or critics? **audience rating on rotten tomatoes??

#I need to create DataFrame with all variables from IMDB api and the movie id from my original DataFrame (so I can test model with ratings)
#Need to make sure movies don't get repeated when adding back into new dataframe with IMDB data
#First thing is getting add data into initial dataframe, need to include columns for each streaming service they are available on

In [2]:
#import modules
import psycopg2 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle 

In [3]:
Movie_list = pd.read_csv('Movies.csv', header = 0)

#for movie in data_1['title']:
    #movie_list.append(movie)



def deleteThe(title):
    title = title.replace(', The', '')
    if title[:-6] == ', The':
        return title.replace(', The', '')
    else:
        return title

def deleteLastplus(title):
    if title[-1] == '+':
        title = title[0:-1]
        return title
    else:
        return title

Movie_list['year'] = Movie_list['title'].str.extract('(\d\d\d\d)', expand=True)
Movie_list['title'] = Movie_list['title'].astype(str).str[:-6]
Movie_list['title'] = Movie_list['title'].str.replace(r"\(.*\)","")
Movie_list['title'] = Movie_list['title'].apply(deleteThe)
Movie_list['title'] = Movie_list['title'].str.replace(" ","+").apply(deleteLastplus)
Movie_list['api'] = Movie_list['title'] + '&y=' + Movie_list['year']

Movie_list

,movieId,title,genres,year,api
0,1,Toy+Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy+Story&y=1995
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji&y=1995
2,3,Grumpier+Old+Men,Comedy|Romance,1995,Grumpier+Old+Men&y=1995
3,4,Waiting+to+Exhale,Comedy|Drama|Romance,1995,Waiting+to+Exhale&y=1995
4,5,Father+of+the+Bride+Part+II,Comedy,1995,Father+of+the+Bride+Part+II&y=1995
5,6,Heat,Action|Crime|Thriller,1995,Heat&y=1995
6,7,Sabrina,Comedy|Romance,1995,Sabrina&y=1995
7,8,Tom+and+Huck,Adventure|Children,1995,Tom+and+Huck&y=1995
8,9,Sudden+Death,Action,1995,Sudden+Death&y=1995
9,10,GoldenEye,Action|Adventure|Thriller,1995,GoldenEye&y=1995


In [4]:
Data_for_model = pd.read_csv('ratings.csv', header = 0)
len(Data_for_model)

20000263

In [5]:
#http://virendra.me/scraping-imdb-with-python/
#https://github.com/richardasaurus/imdb-pie
#https://github.com/jameskang410/scraping-netflix
#https://www.cinesift.com/#/
#http://www.hbo.com/movies/catalog

#what do I need
'''1. movie*
   2. imdb data
   3. list of netflix movies
   4. list of hbo movies
   5. list of hulu movies
   6. list of amazon movies
   7. list of showtime movies'''



'1. movie*\n   2. imdb data\n   3. list of netflix movies\n   4. list of hbo movies\n   5. list of hulu movies\n   6. list of amazon movies\n   7. list of showtime movies'

In [6]:
#showtime 
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from selenium import webdriver
links = ['http://www.sho.com/movies/action', 'http://www.sho.com/movies/action/page/2', \
         'http://www.sho.com/movies/action/page/3', 'http://www.sho.com/movies/comedy', \
         'http://www.sho.com/movies/comedy/page/2', 'http://www.sho.com/movies/comedy/page/3', \
         'http://www.sho.com/movies/comedy/page/4', 'http://www.sho.com/movies/comedy/page/5', \
         'http://www.sho.com/movies/documentary', 'http://www.sho.com/movies/documentary/page/2', \
         'http://www.sho.com/movies/drama', 'http://www.sho.com/movies/drama/page/2', \
         'http://www.sho.com/movies/drama/page/3', 'http://www.sho.com/movies/drama/page/4', \
         'http://www.sho.com/movies/drama/page/5', 'http://www.sho.com/movies/drama/page/6', \
         'http://www.sho.com/movies/drama/page/7', 'http://www.sho.com/movies/family', \
         'http://www.sho.com/movies/horror', 'http://www.sho.com/movies/horror/page/2', \
         'http://www.sho.com/movies/music', 'http://www.sho.com/movies/sci-fi-fantasy', \
         'http://www.sho.com/movies/sci-fi-fantasy/page/2', 'http://www.sho.com/movies/suspense', \
         'http://www.sho.com/movies/suspense/page/2', 'http://www.sho.com/movies/suspense/page/3', \
         'http://www.sho.com/movies/suspense/page/4', 'http://www.sho.com/movies/suspense/page/5', \
         'http://www.sho.com/movies/suspense/page/6', 'http://www.sho.com/movies/suspense/page/7']

showtime = []

for link in links:
    show_movie = requests.get(link)
    show_soup = BeautifulSoup(show_movie.content, "lxml")
    titles = show_soup.findAll(class_='movies-gallery__title')
    for title in titles:
        if title in showtime:
            continue
        else:
            showtime.append([title.text, 'showtime'])

showtime_df = pd.DataFrame(data = showtime, columns = ['title', 'service'])
showtime_df

,title,service
0,11 Blocks,showtime
1,3:10 To Yuma,showtime
2,A.R.C.H.I.E.,showtime
3,The Accidental Spy,showtime
4,After Words,showtime
5,The Alamo (2004),showtime
6,Alone in the Dark,showtime
7,Arctic Tale,showtime
8,Arthur & Merlin,showtime
9,Awaken,showtime


In [7]:
import openpyxl
netflix_movies = []
netflix = openpyxl.load_workbook('NetflixMovies.xlsx')
sheet = netflix.get_sheet_by_name('Movies')
for x in range(1,4393):
    netflix_movies.append([sheet.cell(row=x, column=2).value, 'netflix'])

netflix_df = pd.DataFrame(data = netflix_movies, columns = ['title', 'service'])

//anaconda/lib/python2.7/site-packages/openpyxl/reader/worksheet.py:322: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [8]:
hulu_movies = []
hulu = openpyxl.load_workbook('HuluMovies.xlsx')
sheet = hulu.get_sheet_by_name('Sheet1')
for x in range(2,1090):
    hulu_movies.append([sheet.cell(row=x, column=4).value, 'hulu'])

hulu_df = pd.DataFrame(data = hulu_movies, columns = ['title', 'service'])
hulu_df

,title,service
0,Daddy's Home,hulu
1,Zoolander 2,hulu
2,13 Hours: The Secret Soldiers of Benghazi,hulu
3,Iron Man (2008),hulu
4,Whiskey Tango Foxtrot,hulu
5,Raiders of the Lost Ark,hulu
6,Spider-Man 3,hulu
7,Norm of the North,hulu
8,Baby Boom,hulu
9,Mulan,hulu


In [9]:
amazon_movies = []
amazon = openpyxl.load_workbook('AmazonMovies.xlsx')
sheet = amazon.get_sheet_by_name('Sheet1')
for x in range(2,2537):
    sheet.cell(row=x, column=1).value
    amazon_movies.append(sheet.cell(row=x, column=1).value)

Amazon_Movies = []
for line in amazon_movies:
    if line[0] == " ":
        continue
    else:
        Amazon_Movies.append([line, 'amazon'])

amazon_df = pd.DataFrame(data = Amazon_Movies, columns = ['title', 'service'])
amazon_df

,title,service
0,Whiskey Tango Foxtrot,amazon
1,Fort Tilden,amazon
2,The Dark Horse,amazon
3,Equals,amazon
4,The Russian Woodpecker,amazon
5,Louder Than Bombs,amazon
6,The Sea Of Trees,amazon
7,Green Room,amazon
8,Wiener-Dog,amazon
9,The Lobster,amazon


In [10]:
HBO_movies = []
HBO = openpyxl.load_workbook('HBOMovies.xlsx')
sheet = HBO.get_sheet_by_name('Sheet1')
for x in range(2,345):
    sheet.cell(row=x, column=2).value
    HBO_movies.append([sheet.cell(row=x, column=2).value, 'HBO'])


HBO_df = pd.DataFrame(data = HBO_movies, columns = ['title', 'service'])
HBO_movies



[[u'12 Rounds', 'HBO'],
 [u'12 Years A Slave', 'HBO'],
 [u'127 Hours', 'HBO'],
 [u'17 Again', 'HBO'],
 [u'2 Guns', 'HBO'],
 [u'300: Rise of an Empire', 'HBO'],
 [u'The 33', 'HBO'],
 [42L, 'HBO'],
 [u'47 Ronin', 'HBO'],
 [u'*61', 'HBO'],
 [u'A Good Day to Die Hard', 'HBO'],
 [u'A Lesson Before Dying', 'HBO'],
 [u'A Million Ways to Die in the West', 'HBO'],
 [u'A Nightmare on Elm Street', 'HBO'],
 [u'A Number', 'HBO'],
 [u'A Perfect Getaway', 'HBO'],
 [u'A Private Matter', 'HBO'],
 [u'A Thousand Words', 'HBO'],
 [u'A Very Harold & Kumar 3D Christmas', 'HBO'],
 [u'A Walk Among the Tombstones', 'HBO'],
 [u'The A-Team', 'HBO'],
 [u'Abraham Lincoln: Vampire Hunter', 'HBO'],
 [u'The Adjustment Bureau', 'HBO'],
 [u'Admission', 'HBO'],
 [u'The Affair', 'HBO'],
 [u'Afterburn', 'HBO'],
 [u'Against the Wall', 'HBO'],
 [u'All About Steve', 'HBO'],
 [u'All The Way', 'HBO'],
 [u'Alvin & The Chipmunks: Chipwrecked', 'HBO'],
 [u'Alvin and The Chipmunks: The Road Chip', 'HBO'],
 [u'Alvin and the Chipmun

In [11]:
import openpyxl
tomatoes_movies = []
tomatoes = openpyxl.load_workbook('movie_metadata.xlsx')
sheet = tomatoes.get_sheet_by_name('sheet1')
for x in range(2,5045):
    tomatoes_movies.append([sheet.cell(row=x, column=12).value, ''])
    
for x in tomatoes_movies:
    for y in x:
        if y in ['c', 'd' ]:
            x = x.replace(y, "")
    
tomatoes_df = pd.DataFrame(data = tomatoes_movies, columns = ['title', 'service'])

tomatoes_df


,title,service
0,Avatar,
1,Pirates of the Caribbean: At World's End,
2,Spectre,
3,The Dark Knight Rises,
4,Star Wars: Episode VII - The Force Awakens ...,
5,John Carter,
6,Spider-Man 3,
7,Tangled,
8,Avengers: Age of Ultron,
9,Harry Potter and the Half-Blood Prince,


In [12]:
services = [HBO_df, showtime_df, amazon_df, netflix_df, hulu_df, tomatoes_df]
Movies_services = pd.concat(services)
Movies_services['year'] = Movies_services['title'].str.extract('(\d\d\d\d)', expand=True)
Movies_services['title'] = Movies_services['title'].str.replace(r"\(.*\)","")
Movies_services['title'] = Movies_services['title'].str.replace(" ","+")
Movies_services['movieId'] = [x for x in range(140000, 152622)]
print len(Movies_services)
print Movies_services
#delete titles that are null values


12622
                                   title service year  movieId
0                              12+Rounds     HBO  NaN   140000
1                       12+Years+A+Slave     HBO  NaN   140001
2                              127+Hours     HBO  NaN   140002
3                               17+Again     HBO  NaN   140003
4                                 2+Guns     HBO  NaN   140004
5                 300:+Rise+of+an+Empire     HBO  NaN   140005
6                                 The+33     HBO  NaN   140006
7                                    NaN     HBO  NaN   140007
8                               47+Ronin     HBO  NaN   140008
9                                    *61     HBO  NaN   140009
10                A+Good+Day+to+Die+Hard     HBO  NaN   140010
11                 A+Lesson+Before+Dying     HBO  NaN   140011
12     A+Million+Ways+to+Die+in+the+West     HBO  NaN   140012
13             A+Nightmare+on+Elm+Street     HBO  NaN   140013
14                              A+Number     HBO 

In [13]:
Movies_services['api'] = Movies_services['title'] + '&y='
Complete_df = pd.merge(Movie_list, Movies_services, how='outer')
import pickle
with open('complete_df.pkl', 'w') as picklefile:
    pickle.dump(Complete_df, picklefile)
Complete_df

//anaconda/lib/python2.7/site-packages/pandas/tools/merge.py:714: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  rlab = rizer.factorize(rk)


,movieId,title,genres,year,api,service
0,1.0,Toy+Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy+Story&y=1995,NaN
1,2.0,Jumanji,Adventure|Children|Fantasy,1995,Jumanji&y=1995,NaN
2,3.0,Grumpier+Old+Men,Comedy|Romance,1995,Grumpier+Old+Men&y=1995,NaN
3,4.0,Waiting+to+Exhale,Comedy|Drama|Romance,1995,Waiting+to+Exhale&y=1995,NaN
4,5.0,Father+of+the+Bride+Part+II,Comedy,1995,Father+of+the+Bride+Part+II&y=1995,NaN
5,6.0,Heat,Action|Crime|Thriller,1995,Heat&y=1995,NaN
6,7.0,Sabrina,Comedy|Romance,1995,Sabrina&y=1995,NaN
7,8.0,Tom+and+Huck,Adventure|Children,1995,Tom+and+Huck&y=1995,NaN
8,9.0,Sudden+Death,Action,1995,Sudden+Death&y=1995,NaN
9,10.0,GoldenEye,Action|Adventure|Thriller,1995,GoldenEye&y=1995,NaN


In [14]:
#list of column names from one dictionary of data
#Werner - Gekotzt wird später

attributes = []
from urllib2 import urlopen
movies = urlopen("http://www.omdbapi.com/?t=Vampire's+Kiss&y=1988&plot=short&r=json&tomatoes=true")
response = movies.read()
response_dict = eval(response)
for key in response_dict.keys():
    attributes.append(key)
print response_dict

{'Plot': "After an encounter with a neck-biter, a publishing executive thinks that he's turning into a vampire.", 'Rated': 'R', 'Language': 'English, German', 'Title': "Vampire's Kiss", 'Country': 'USA', 'tomatoMeter': '59', 'Writer': 'Joseph Minion', 'tomatoURL': 'http://www.rottentomatoes.com/m/vampires_kiss/', 'BoxOffice': 'N/A', 'tomatoUserRating': '3.0', 'Production': 'N/A', 'Actors': 'Nicolas Cage, Maria Conchita Alonso, Jennifer Beals, Elizabeth Ashley', 'tomatoFresh': '13', 'Runtime': '103 min', 'Type': 'movie', 'imdbVotes': '9,775', 'Website': 'N/A', 'tomatoConsensus': 'N/A', 'tomatoReviews': '22', 'tomatoImage': 'rotten', 'Poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMTM1MDAyMDYxMV5BMl5BanBnXkFtZTcwNDQwMzc3NA@@._V1_SX300.jpg', 'tomatoRotten': '9', 'Metascore': '31', 'Response': 'True', 'Director': 'Robert Bierman', 'Released': '02 Jun 1989', 'tomatoUserReviews': '8864', 'Awards': '1 win & 2 nominations.', 'tomatoRating': '5.9', 'Year': '1988', 'Genre': 'Come

In [15]:
#to speed up maybe manually type in all of the different names of lists

#from urllib2 import urlopen
#for attribute in attributes:
    #name = attribute
    #attribute = []
    #for item, row in abr_df.iterrows():
        #try:
            #api = row['api']
            #movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
            #results = movies.read()
            #response_dict = eval(results)
            #place_hold = response_dict[name]
            #attribute.append(place_hold)
        #except:
            #attribute.append(None)
            #pass
    #abr_df[name] = attribute
#abr_df

In [16]:
#some parts of dataframe don't have api column values, also print out error rows to further clean
#to speed up maybe manually type in all of the different names of lists
with open('complete_df.pkl', 'r') as picklefile:
    Complete_df = pickle.load(picklefile)

from urllib2 import urlopen
def pop_df(data):
    Plot = []
    Rated = []
    Language = []
    Title = []
    Country = []
    tomatoMeter = []
    Writer = []
    tomatoURL = []
    BoxOffice = []
    tomatoUserRating = []
    Production = []
    Actors = []
    tomatoFresh = []
    Runtime = []
    Type = []
    imdbVotes = []
    Website = []
    tomatoConsensus = []
    tomatoReviews = []
    tomatoImage = []
    Poster = []
    tomatoRotten = []
    Metascore = []
    Response = []
    Director = []
    Released = []
    tomatoUserReviews = []
    Awards = []
    tomatoRating = []
    Year = []
    Genre = []
    tomatoUserMeter = []
    DVD = []
    imdbRating = []
    imdbID = []
    for item, row in data.iterrows():
        try:
            api = row['api']
            movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
            results = movies.read()
            response_dict = eval(results)
            rated = response_dict['Rated']
            Rated.append(rated)
            plot = response_dict['Plot']
            Plot.append(plot)
            language = response_dict['Language']
            Language.append(language)
            title = response_dict['Title']
            Title.append(title)
            country = response_dict['Country']
            Country.append(country)
            tm = response_dict['tomatoMeter']
            tomatoMeter.append(tm)
            wr = response_dict['Writer']
            Writer.append(wr)
            turl = response_dict['tomatoURL']
            tomatoURL.append(turl)
            BO = response_dict['BoxOffice']
            BoxOffice.append(BO)
            tUR = response_dict['tomatoUserRating']
            tomatoUserRating.append(tUR)
            pro = response_dict['Production']
            Production.append(pro)
            act = response_dict['Actors']
            Actors.append(act)
            tf = response_dict['tomatoFresh']
            tomatoFresh.append(tf)
            rt = response_dict['Runtime']
            Runtime.append(rt)
            ty = response_dict['Type']
            Type.append(ty)
            imdbV = response_dict['imdbVotes']
            imdbVotes.append(imdbV)
            tomC = response_dict['tomatoConsensus']
            tomatoConsensus.append(tomC)
            tomR = response_dict['tomatoReviews']
            tomatoReviews.append(tomR)
            tomI = response_dict['tomatoImage']
            tomatoImage.append(tomI)
            post = response_dict['Poster']
            Poster.append(post)
            tomRot = response_dict['tomatoRotten']
            tomatoRotten.append(tomRot)
            meta = response_dict['Metascore']
            Metascore.append(meta)
            resp = response_dict['Response']
            Response.append(resp)
            dire = response_dict['Director']
            Director.append(dire)
            rele = response_dict['Released']
            Released.append(rele)
            tomURev = response_dict['tomatoUserReviews']
            tomatoUserReviews.append(tomURev)
            aw = response_dict['Awards']
            Awards.append(aw)
            tomRat = response_dict['tomatoRating']
            tomatoRating.append(tomRat)
            year = response_dict['Year']
            Year.append(year)
            gen = response_dict['Genre']
            Genre.append(gen)
            tomUM = response_dict['tomatoUserMeter']
            tomatoUserMeter.append(tomUM)
            dvd = response_dict['DVD']
            DVD.append(dvd)
            imdbRat = response_dict['imdbRating']
            imdbRating.append(imdbRat)
            imid = response_dict['imdbID']
            imdbID.append(imid)
            web = response_dict['Website']
            Website.append(web)
        except:
            x = row['api']
            print x
            Plot.append(None)
            Rated.append(None)
            Language.append(None)
            Title.append(None)
            Country.append(None)
            tomatoMeter.append(None)
            Writer.append(None)
            tomatoURL.append(None)
            BoxOffice.append(None)
            tomatoUserRating.append(None)
            Production.append(None)
            Actors.append(None)
            tomatoFresh.append(None)
            Runtime.append(None)
            Type.append(None)
            imdbVotes.append(None)
            Website.append(None)
            tomatoConsensus.append(None)
            tomatoReviews.append(None)
            tomatoImage.append(None)
            Poster.append(None)
            tomatoRotten.append(None)
            Metascore.append(None)
            Response.append(None)
            Director.append(None)
            Released.append(None)
            tomatoUserReviews.append(None)
            Awards.append(None)
            tomatoRating.append(None)
            Year.append(None)
            Genre.append(None)
            tomatoUserMeter.append(None)
            DVD.append(None)
            imdbRating.append(None)
            imdbID.append(None)
    data['Plot'] = Plot
    data['Rated'] = Rated
    data['Language'] = Language
    data['Title'] = Title
    data['Country'] = Country
    data['tomatoMeter'] = tomatoMeter
    data['Writer'] = Writer
    data['tomatoURL'] = tomatoURL
    data['BoxOffice'] = BoxOffice
    data['tomatoUserRating'] = tomatoUserRating
    data['Production'] = Production
    data['Actors'] = Actors
    data['tomatoFresh'] = tomatoFresh
    data['Runtime'] = Runtime
    data['Type'] = Type
    data['imdbVotes'] = imdbVotes
    data['Website'] = Website
    data['tomatoConsensus'] = tomatoConsensus
    data['tomatoReviews'] = tomatoReviews
    data['tomatoImage'] = tomatoImage
    data['Poster'] = Poster
    data['tomatoRotten'] = tomatoRotten
    data['Metascore'] = Metascore
    data['Response'] = Response
    data['Director'] = Director
    data['Released'] = Released
    data['tomatoUserReviews'] = tomatoUserReviews
    data['Awards'] = Awards
    data['tomatoRating '] = tomatoRating 
    data['Year'] = Year
    data['Genre'] = Genre
    data['tomatoUserMeter'] = tomatoUserMeter
    data['DVD'] = DVD
    data['imdbRating'] = imdbRating
    data['imdbID'] = imdbID
    return data
first_thou = Complete_df[0:1000]
sec_thou = Complete_df[1000:2000]
thi_thou = Complete_df[3000:4000]
four_thou = Complete_df[4000:5000]
fif_thou = Complete_df[5000:6000]
six_thou = Complete_df[6000:7000]
sev_thou = Complete_df[7000:8000]
eig_thou = Complete_df[8000:9000]
nin_thou = Complete_df[9000:10000]

pop_df(first_thou)



Misérables,+Les&y=1995
In+the+Bleak+Midwinter&y=1995
Catwalk&y=1996
Jupiter's+Wife&y=1994
Awfully+Big+Adventure,+An&y=1995
Beauty+of+the+Day+&y=1967
Mute+Witness&y=1994
To+Wong+Foo,+Thanks+for+Everything!+Julie+Newmar&y=1995
Walk+in+the+Clouds,+A&y=1995
Don+Juan+DeMarco&y=1995
Goofy+Movie,+A&y=1995
Farinelli:+il+castrato&y=1994
Houseguest&y=1994
Heavyweights+&y=1995
Kid+in+King+Arthur's+Court,+A&y=1995
Little+Princess,+A&y=1995
Enfer,+L'&y=1994
My+Crazy+Life+&y=1993
Beyond+Bedlam&y=1993
Pyromaniac's+Love+Story,+A&y=1995
Swimming+with+Sharks&y=1995
National+Lampoon's+Senior+Trip&y=1995
Tales+from+the+Crypt+Presents:+Demon+Knight&y=1995
Backbeat&y=1993
Wes+Craven's+New+Nightmare+&y=1994
Red+Rock+West&y=1992
Richie+Rich&y=1994
Man+of+No+Importance,+A&y=1994
Low+Down+Dirty+Shame,+A&y=1994
Boys+Life&y=1995
Colonel+Chabert,+Le&y=1994
Desert+Winds&y=1995
Open+Season&y=1996
Two+Crimes+&y=1995
Highlander+III:+The+Sorcerer+&y=1994
In+the+Mouth+of+Madness&y=1995
Martin+Lawrence:+You+So+Crazy&y=19

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

,movieId,title,genres,year,api,service,Plot,Rated,Language,Title,...,Released,tomatoUserReviews,Awards,tomatoRating,Year,Genre,tomatoUserMeter,DVD,imdbRating,imdbID
0,1.0,Toy+Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy+Story&y=1995,NaN,A cowboy doll is profoundly threatened and jea...,G,English,Toy Story,...,22 Nov 1995,1106743,Nominated for 3 Oscars. Another 23 wins & 18 n...,9.0,1995,"Animation, Adventure, Comedy",92,N/A,8.3,tt0114709
1,2.0,Jumanji,Adventure|Children|Fantasy,1995,Jumanji&y=1995,NaN,When two kids find and play a magical board ga...,PG,"English, French",Jumanji,...,15 Dec 1995,973907,4 wins & 9 nominations.,5.6,1995,"Action, Adventure, Family",62,N/A,6.9,tt0113497
2,3.0,Grumpier+Old+Men,Comedy|Romance,1995,Grumpier+Old+Men&y=1995,NaN,John and Max resolve to save their beloved bai...,PG-13,English,Grumpier Old Men,...,22 Dec 1995,70062,2 wins & 2 nominations.,4.2,1995,"Comedy, Romance",62,N/A,6.6,tt0113228
3,4.0,Waiting+to+Exhale,Comedy|Drama|Romance,1995,Waiting+to+Exhale&y=1995,NaN,"Based on Terry McMillan's novel, this film fol...",R,English,Waiting to Exhale,...,22 Dec 1995,38176,8 wins & 8 nominations.,5.6,1995,"Comedy, Drama, Romance",79,N/A,5.7,tt0114885
4,5.0,Father+of+the+Bride+Part+II,Comedy,1995,Father+of+the+Bride+Part+II&y=1995,NaN,George Banks must deal not only with the pregn...,PG,English,Father of the Bride Part II,...,08 Dec 1995,202967,Nominated for 1 Golden Globe. Another 1 win & ...,5.3,1995,"Comedy, Family, Romance",60,N/A,5.9,tt0113041
5,6.0,Heat,Action|Crime|Thriller,1995,Heat&y=1995,NaN,A group of professional bank robbers start to ...,R,"English, Spanish",Heat,...,15 Dec 1995,220166,12 nominations.,7.8,1995,"Action, Crime, Drama",94,N/A,8.2,tt0113277
6,7.0,Sabrina,Comedy|Romance,1995,Sabrina&y=1995,NaN,An ugly duckling having undergone a remarkable...,PG,"English, French",Sabrina,...,15 Dec 1995,52123,Nominated for 2 Oscars. Another 2 wins & 4 nom...,6.0,1995,"Comedy, Drama",64,N/A,6.3,tt0114319
7,8.0,Tom+and+Huck,Adventure|Children,1995,Tom+and+Huck&y=1995,NaN,Tom and Huck witness Injun Joe's killing of Do...,PG,English,Tom and Huck,...,22 Dec 1995,39473,1 win & 5 nominations.,4.2,1995,"Adventure, Comedy, Drama",41,N/A,5.6,tt0112302
8,9.0,Sudden+Death,Action,1995,Sudden+Death&y=1995,NaN,A former fireman takes on a group of terrorist...,R,English,Sudden Death,...,22 Dec 1995,34159,N/A,5.1,1995,"Action, Crime, Thriller",38,N/A,5.7,tt0114576
9,10.0,GoldenEye,Action|Adventure|Thriller,1995,GoldenEye&y=1995,NaN,James Bond teams up with the lone survivor of ...,PG-13,"English, Russian, Spanish",GoldenEye,...,17 Nov 1995,284930,Nominated for 2 BAFTA Film Awards. Another 2 w...,7.0,1995,"Action, Adventure, Thriller",83,N/A,7.2,tt0113189


In [17]:
#some parts of dataframe don't have api column values, also print out error rows to further clean
#to speed up maybe manually type in all of the different names of lists
from urllib2 import urlopen
def pop_df(data):
    Plot = []
    Rated = []
    Language = []
    Title = []
    Country = []
    tomatoMeter = []
    Writer = []
    tomatoURL = []
    BoxOffice = []
    tomatoUserRating = []
    Production = []
    Actors = []
    tomatoFresh = []
    Runtime = []
    Type = []
    imdbVotes = []
    Website = []
    tomatoConsensus = []
    tomatoReviews = []
    tomatoImage = []
    Poster = []
    tomatoRotten = []
    Metascore = []
    Response = []
    Director = []
    Released = []
    tomatoUserReviews = []
    Awards = []
    tomatoRating = []
    Year = []
    Genre = []
    tomatoUserMeter = []
    DVD = []
    imdbRating = []
    imdbID = []
    for item, row in data.iterrows():
        try:
            api = row['api']
            movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
            results = movies.read()
            response_dict = eval(results)
            rated = response_dict['Rated']
            Rated.append(rated)
            plot = response_dict['Plot']
            Plot.append(plot)
            language = response_dict['Language']
            Language.append(language)
            title = response_dict['Title']
            Title.append(title)
            country = response_dict['Country']
            Country.append(country)
            tm = response_dict['tomatoMeter']
            tomatoMeter.append(tm)
            wr = response_dict['Writer']
            Writer.append(wr)
            turl = response_dict['tomatoURL']
            tomatoURL.append(turl)
            BO = response_dict['BoxOffice']
            BoxOffice.append(BO)
            tUR = response_dict['tomatoUserRating']
            tomatoUserRating.append(tUR)
            pro = response_dict['Production']
            Production.append(pro)
            act = response_dict['Actors']
            Actors.append(act)
            tf = response_dict['tomatoFresh']
            tomatoFresh.append(tf)
            rt = response_dict['Runtime']
            Runtime.append(rt)
            ty = response_dict['Type']
            Type.append(ty)
            imdbV = response_dict['imdbVotes']
            imdbVotes.append(imdbV)
            tomC = response_dict['tomatoConsensus']
            tomatoConsensus.append(tomC)
            tomR = response_dict['tomatoReviews']
            tomatoReviews.append(tomR)
            tomI = response_dict['tomatoImage']
            tomatoImage.append(tomI)
            post = response_dict['Poster']
            Poster.append(post)
            tomRot = response_dict['tomatoRotten']
            tomatoRotten.append(tomRot)
            meta = response_dict['Metascore']
            Metascore.append(meta)
            resp = response_dict['Response']
            Response.append(resp)
            dire = response_dict['Director']
            Director.append(dire)
            rele = response_dict['Released']
            Released.append(rele)
            tomURev = response_dict['tomatoUserReviews']
            tomatoUserReviews.append(tomURev)
            aw = response_dict['Awards']
            Awards.append(aw)
            tomRat = response_dict['tomatoRating']
            tomatoRating.append(tomRat)
            year = response_dict['Year']
            Year.append(year)
            gen = response_dict['Genre']
            Genre.append(gen)
            tomUM = response_dict['tomatoUserMeter']
            tomatoUserMeter.append(tomUM)
            dvd = response_dict['DVD']
            DVD.append(dvd)
            imdbRat = response_dict['imdbRating']
            imdbRating.append(imdbRat)
            imid = response_dict['imdbID']
            imdbID.append(imid)
            web = response_dict['Website']
            Website.append(web)
        except:
            x = row['api']
            print x
            Plot.append(None)
            Rated.append(None)
            Language.append(None)
            Title.append(None)
            Country.append(None)
            tomatoMeter.append(None)
            Writer.append(None)
            tomatoURL.append(None)
            BoxOffice.append(None)
            tomatoUserRating.append(None)
            Production.append(None)
            Actors.append(None)
            tomatoFresh.append(None)
            Runtime.append(None)
            Type.append(None)
            imdbVotes.append(None)
            Website.append(None)
            tomatoConsensus.append(None)
            tomatoReviews.append(None)
            tomatoImage.append(None)
            Poster.append(None)
            tomatoRotten.append(None)
            Metascore.append(None)
            Response.append(None)
            Director.append(None)
            Released.append(None)
            tomatoUserReviews.append(None)
            Awards.append(None)
            tomatoRating.append(None)
            Year.append(None)
            Genre.append(None)
            tomatoUserMeter.append(None)
            DVD.append(None)
            imdbRating.append(None)
            imdbID.append(None)
    data['Plot'] = Plot
    data['Rated'] = Rated
    data['Language'] = Language
    data['Title'] = Title
    data['Country'] = Country
    data['tomatoMeter'] = tomatoMeter
    data['Writer'] = Writer
    data['tomatoURL'] = tomatoURL
    data['BoxOffice'] = BoxOffice
    data['tomatoUserRating'] = tomatoUserRating
    data['Production'] = Production
    data['Actors'] = Actors
    data['tomatoFresh'] = tomatoFresh
    data['Runtime'] = Runtime
    data['Type'] = Type
    data['imdbVotes'] = imdbVotes
    data['Website'] = Website
    data['tomatoConsensus'] = tomatoConsensus
    data['tomatoReviews'] = tomatoReviews
    data['tomatoImage'] = tomatoImage
    data['Poster'] = Poster
    data['tomatoRotten'] = tomatoRotten
    data['Metascore'] = Metascore
    data['Response'] = Response
    data['Director'] = Director
    data['Released'] = Released
    data['tomatoUserReviews'] = tomatoUserReviews
    data['Awards'] = Awards
    data['tomatoRating '] = tomatoRating 
    data['Year'] = Year
    data['Genre'] = Genre
    data['tomatoUserMeter'] = tomatoUserMeter
    data['DVD'] = DVD
    data['imdbRating'] = imdbRating
    data['imdbID'] = imdbID
    return data
first_thou = Complete_df[0:10]
sec_thou = Complete_df[1000:2000]
thi_thou = Complete_df[3000:4000]
four_thou = Complete_df[4000:5000]
fif_thou = Complete_df[5000:6000]
six_thou = Complete_df[6000:7000]
sev_thou = Complete_df[7000:8000]
eig_thou = Complete_df[8000:9000]
nin_thou = Complete_df[9000:10000]



In [18]:
#some parts of dataframe don't have api column values, also print out error rows to further clean
#to speed up maybe manually type in all of the different names of lists
abr_df = Complete_df[4000:5000]
from urllib2 import urlopen
def pop_df(data):
    Plot = []
    Rated = []
    Language = []
    Title = []
    Country = []
    tomatoMeter = []
    Writer = []
    tomatoURL = []
    BoxOffice = []
    tomatoUserRating = []
    Production = []
    Actors = []
    tomatoFresh = []
    Runtime = []
    Type = []
    imdbVotes = []
    Website = []
    tomatoConsensus = []
    tomatoReviews = []
    tomatoImage = []
    Poster = []
    tomatoRotten = []
    Metascore = []
    Response = []
    Director = []
    Released = []
    tomatoUserReviews = []
    Awards = []
    tomatoRating = []
    Year = []
    Genre = []
    tomatoUserMeter = []
    DVD = []
    imdbRating = []
    imdbID = []
    for item, row in data.iterrows():
        try:
            api = row['api']
            movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
            results = movies.read()
            response_dict = eval(results)
            try:
                rated = response_dict['Rated']
                Rated.append(rated)
            except:
                Rated.append(None)
            try:
                plot = response_dict['Plot']
                Plot.append(plot)
            except:
                Plot.append(None)
            try:
                language = response_dict['Language']
                Language.append(language)
            except:
                Language.append(None)
            try:
                title = response_dict['Title']
                Title.append(title)
            except:
                Title.append(None)
            try:
                country = response_dict['Country']
                Country.append(country)
            except:
                Country.append(None)
            try:
                tm = response_dict['tomatoMeter']
                tomatoMeter.append(tm)
            except:
                tomatoMeter.append(None)
            try:
                wr = response_dict['Writer']
                Writer.append(wr)
            except:
                Writer.append(None)
            try:
                turl = response_dict['tomatoURL']
                tomatoURL.append(turl)
            except:
                tomatoURL.append(None)
            try:
                BO = response_dict['BoxOffice']
                BoxOffice.append(BO)
            except:
                BoxOffice.append(None)
            try:
                tUR = response_dict['tomatoUserRating']
                tomatoUserRating.append(tUR)
            except:
                tomatoUserRating.append(None)
            try:
                pro = response_dict['Production']
                Production.append(pro)
            except:
                Production.append(None)
            try:  
                act = response_dict['Actors']
                Actors.append(act)
            except: 
                Actors.append(None)
            try:
                tf = response_dict['tomatoFresh']
                tomatoFresh.append(tf)
            except:
                tomatoFresh.append(None)
            try:
                rt = response_dict['Runtime']
                Runtime.append(rt)
            except:
                Runtime.append(None)
            try:   
                ty = response_dict['Type']
                Type.append(ty)
            except:
                Type.append(None)
            try:
                imdbV = response_dict['imdbVotes']
                imdbVotes.append(imdbV)
            except: 
                imdbVotes.append(None)
            try: 
                tomC = response_dict['tomatoConsensus']
                tomatoConsensus.append(tomC)
            except: 
                tomatoConsensus.append(None)
            try:
                tomR = response_dict['tomatoReviews']
                tomatoReviews.append(tomR)
            except: 
                tomatoReviews.append(None)
            try:
                tomI = response_dict['tomatoImage']
                tomatoImage.append(tomI)
            except:
                omatoImage.append(None)
            try:
                post = response_dict['Poster']
                Poster.append(post)
            except: 
                Poster.append(None)
            try:
                tomRot = response_dict['tomatoRotten']
                tomatoRotten.append(tomRot)
            except:
                tomatoRotten.append(None)
            try:
                meta = response_dict['Metascore']
                Metascore.append(meta)
            except: 
                Metascore.append(None)
            try: 
                resp = response_dict['Response']
                Response.append(resp)
            except:
                Response.append(None)
            try:
                dire = response_dict['Director']
                Director.append(dire)
            except:
                Director.append(None)
            try:
                rele = response_dict['Released']
                Released.append(rele)
            except:
                eleased.append(None)
            try: 
                tomURev = response_dict['tomatoUserReviews']
                tomatoUserReviews.append(tomURev)
            except:
                tomatoUserReviews.append(None)
            try:
                aw = response_dict['Awards']
                Awards.append(aw)
            except:
                Awards.append(None)
            try:  
                tomRat = response_dict['tomatoRating']
                tomatoRating.append(tomRat)
            except:
                tomatoRating.append(None)
            try:
                year = response_dict['Year']
                Year.append(year)
            except:
                Year.append(None)
            try:
                gen = response_dict['Genre']
                Genre.append(gen)
            except:
                Genre.append(None)
            try:
                tomUM = response_dict['tomatoUserMeter']
                tomatoUserMeter.append(tomUM)
            except:
                tomatoUserMeter.append(None)
            try:   
                dvd = response_dict['DVD']
                DVD.append(dvd)
            except:
                DVD.append(None)
            try:
                imdbRat = response_dict['imdbRating']
                imdbRating.append(imdbRat)
            except:
                imdbRating.append(None)
            try:
                imid = response_dict['imdbID']
                imdbID.append(imid)
            except: 
                imdbID.append(None)
            try:
                web = response_dict['Website']
                Website.append(web)
            except:
                Website.append(None)
        except:
            Plot.append(None)
            Rated.append(None)
            Language.append(None)
            Title.append(None)
            Country.append(None)
            tomatoMeter.append(None)
            Writer.append(None)
            tomatoURL.append(None)
            BoxOffice.append(None)
            tomatoUserRating.append(None)
            Production.append(None)
            Actors.append(None)
            tomatoFresh.append(None)
            Runtime.append(None)
            Type.append(None)
            imdbVotes.append(None)
            Website.append(None)
            tomatoConsensus.append(None)
            tomatoReviews.append(None)
            tomatoImage.append(None)
            Poster.append(None)
            tomatoRotten.append(None)
            Metascore.append(None)
            Response.append(None)
            Director.append(None)
            Released.append(None)
            tomatoUserReviews.append(None)
            Awards.append(None)
            tomatoRating.append(None)
            Year.append(None)
            Genre.append(None)
            tomatoUserMeter.append(None)
            DVD.append(None)
            imdbRating.append(None)
            imdbID.append(None)
    data['Plot'] = Plot
    data['Rated'] = Rated
    data['Language'] = Language
    data['Title'] = Title
    data['Country'] = Country
    data['tomatoMeter'] = tomatoMeter
    data['Writer'] = Writer
    data['tomatoURL'] = tomatoURL
    data['BoxOffice'] = BoxOffice
    data['tomatoUserRating'] = tomatoUserRating
    data['Production'] = Production
    data['Actors'] = Actors
    data['tomatoFresh'] = tomatoFresh
    data['Runtime'] = Runtime
    data['Type'] = Type
    data['imdbVotes'] = imdbVotes
    data['Website'] = Website
    data['tomatoConsensus'] = tomatoConsensus
    data['tomatoReviews'] = tomatoReviews
    data['tomatoImage'] = tomatoImage
    data['Poster'] = Poster
    data['tomatoRotten'] = tomatoRotten
    data['Metascore'] = Metascore
    data['Response'] = Response
    data['Director'] = Director
    data['Released'] = Released
    data['tomatoUserReviews'] = tomatoUserReviews
    data['Awards'] = Awards
    data['tomatoRating '] = tomatoRating 
    data['Year'] = Year
    data['Genre'] = Genre
    data['tomatoUserMeter'] = tomatoUserMeter
    data['DVD'] = DVD
    data['imdbRating'] = imdbRating
    data['imdbID'] = imdbID
    return data

pop_df(abr_df)


ValueError: Length of values does not match length of index

In [ ]:
pop_df(sec_thou)

In [ ]:
pop_df(thi_thou)

In [ ]:
pop_df(four_thou)

In [ ]:
pop_df(fif_thou)

In [ ]:
pop_df(six_thou)

In [ ]:
pop_df(sev_thou)

In [ ]:
pop_df(eig_thou)

In [ ]:
pop_df(nin_thou)

In [ ]:
#some parts of dataframe don't have api column values, also print out error rows to further clean
#to speed up maybe manually type in all of the different names of lists
print len(Complete_df)
Complete_df1 = Complete_df[38000:38010]
Complete_df2 = Complete_df[10000:20000]
Complete_df3 = Complete_df[20000:30000]
Complete_df4 = Complete_df[30000:39900]

from urllib2 import urlopen
Plot = []
Rated = []
Language = []
Title = []
Country = []
tomatoMeter = []
Writer = []
tomatoURL = []
BoxOffice = []
tomatoUserRating = []
Production = []
Actors = []
tomatoFresh = []
Runtime = []
Type = []
imdbVotes = []
Website = []
tomatoConsensus = []
tomatoReviews = []
tomatoImage = []
Poster = []
tomatoRotten = []
Metascore = []
Response = []
Director = []
Released = []
tomatoUserReviews = []
Awards = []
tomatoRating = []
Year = []
Genre = []
tomatoUserMeter = []
DVD = []
imdbRating = []
imdbID = []

for item, row in data.iterrows():
    try:
        api = row['api']
        movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
        results = movies.read()
        response_dict = eval(results)
        rated = response_dict['Rated']
        Rated.append(rated)
        plot = response_dict['Plot']
        Plot.append(plot)
        language = response_dict['Language']
        Language.append(language)
        title = response_dict['Title']
        Title.append(title)
        country = response_dict['Country']
        Country.append(country)
        tm = response_dict['tomatoMeter']
        tomatoMeter.append(tm)
        wr = response_dict['Writer']
        Writer.append(wr)
        turl = response_dict['tomatoURL']
        tomatoURL.append(turl)
        BO = response_dict['BoxOffice']
        BoxOffice.append(BO)
        tUR = response_dict['tomatoUserRating']
        tomatoUserRating.append(tUR)
        pro = response_dict['Production']
        Production.append(pro)
        act = response_dict['Actors']
        Actors.append(act)
        tf = response_dict['tomatoFresh']
        tomatoFresh.append(tf)
        rt = response_dict['Runtime']
        Runtime.append(rt)
        ty = response_dict['Type']
        Type.append(ty)
        imdbV = response_dict['imdbVotes']
        imdbVotes.append(imdbV)
        tomC = response_dict['tomatoConsensus']
        tomatoConsensus.append(tomC)
        tomR = response_dict['tomatoReviews']
        tomatoReviews.append(tomR)
        tomI = response_dict['tomatoImage']
        tomatoImage.append(tomI)
        post = response_dict['Poster']
        Poster.append(post)
        tomRot = response_dict['tomatoRotten']
        tomatoRotten.append(tomRot)
        meta = response_dict['Metascore']
        Metascore.append(meta)
        resp = response_dict['Response']
        Response.append(resp)
        dire = response_dict['Director']
        Director.append(dire)
        rele = response_dict['Released']
        Released.append(rele)
        tomURev = response_dict['tomatoUserReviews']
        tomatoUserReviews.append(tomURev)
        aw = response_dict['Awards']
        Awards.append(aw)
        tomRat = response_dict['tomatoRating']
        tomatoRating.append(tomRat)
        year = response_dict['Year']
        Year.append(year)
        gen = response_dict['Genre']
        Genre.append(gen)
        tomUM = response_dict['tomatoUserMeter']
        tomatoUserMeter.append(tomUM)
        dvd = response_dict['DVD']
        DVD.append(dvd)
        imdbRat = response_dict['imdbRating']
        imdbRating.append(imdbRat)
        imid = response_dict['imdbID']
        imdbID.append(imid)
        web = response_dict['Website']
        Website.append(web)
    except:
        Plot.append(None)
        Rated.append(None)
        Language.append(None)
        Title.append(None)
        Country.append(None)
        tomatoMeter.append(None)
        Writer.append(None)
        tomatoURL.append(None)
        BoxOffice.append(None)
        tomatoUserRating.append(None)
        Production.append(None)
        Actors.append(None)
        tomatoFresh.append(None)
        Runtime.append(None)
        Type.append(None)
        imdbVotes.append(None)
        Website.append(None)
        tomatoConsensus.append(None)
        tomatoReviews.append(None)
        tomatoImage.append(None)
        Poster.append(None)
        tomatoRotten.append(None)
        Metascore.append(None)
        Response.append(None)
        Director.append(None)
        Released.append(None)
        tomatoUserReviews.append(None)
        Awards.append(None)
        tomatoRating.append(None)
        Year.append(None)
        Genre.append(None)
        tomatoUserMeter.append(None)
        DVD.append(None)
        imdbRating.append(None)
        imdbID.append(None)
        pass


Complete_df1['Plot'] = Plot
Complete_df1['Rated'] = Rated
Complete_df1['Language'] = Language
Complete_df1['Title'] = Title
Complete_df1['Country'] = Country
Complete_df1['tomatoMeter'] = tomatoMeter
Complete_df1['Writer'] = Writer
Complete_df1['tomatoURL'] = tomatoURL
Complete_df1['BoxOffice'] = BoxOffice
Complete_df1['tomatoUserRating'] = tomatoUserRating
Complete_df1['Production'] = Production
Complete_df1['Actors'] = Actors
Complete_df1['tomatoFresh'] = tomatoFresh
Complete_df1['Runtime'] = Runtime
Complete_df1['Type'] = Type
Complete_df1['imdbVotes'] = imdbVotes
Complete_df1['Website'] = Website
Complete_df1['tomatoConsensus'] = tomatoConsensus
Complete_df1['tomatoReviews'] = tomatoReviews
Complete_df1['tomatoImage'] = tomatoImage
Complete_df1['Poster'] = Poster
Complete_df1['tomatoRotten'] = tomatoRotten
Complete_df1['Metascore'] = Metascore
Complete_df1['Response'] = Response
Complete_df1['Director'] = Director
Complete_df1['Released'] = Released
Complete_df1['tomatoUserReviews'] = tomatoUserReviews
Complete_df1['Awards'] = Awards
Complete_df1['tomatoRating '] = tomatoRating 
Complete_df1['Year'] = Year
Complete_df1['Genre'] = Genre
Complete_df1['tomatoUserMeter'] = tomatoUserMeter
Complete_df1['DVD'] = DVD
Complete_df1['imdbRating'] = imdbRating
Complete_df1['imdbID'] = imdbID   

Complete_df1

In [ ]:
#to speed up maybe manually type in all of the different names of lists

from urllib2 import urlopen
for attribute in attributes:
    name = attribute
    attribute = []
    for item, row in abr_df.iterrows():
        try:
            api = row['api']
            movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
            results = movies.read()
            response_dict = eval(results)
            place_hold = response_dict[name]
            attribute.append(place_hold)
        except:
            attribute.append(None)
            pass
    abr[name] = attribute
abr_df
    
       
    

In [ ]:
#to speed up maybe manually type in all of the different names of lists

#from urllib2 import urlopen
#for attribute in attributes:
    #name = attribute
    #attribute = []
    #for item, row in abr_df.iterrows():
        #try:
            #api = row['api']
            #movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
            #results = movies.read()
            #response_dict = eval(results)
            #place_hold = response_dict[name]
            #attribute.append(place_hold)
        #except:
            #attribute.append(None)
            #pass
    #abr_df[name] = attribute
#abr_df